In [2]:
# --- Celda 1 ---
import pandas as pd
import numpy as np
import logging
import time

# Configuración de logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(message)s",
    datefmt="%H:%M:%S"
)
logger = logging.getLogger(__name__)

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 140)

# 1) Lectura (NO olvides el nombre del archivo)
t0 = time.time()
df = pd.read_csv("base_dups.csv")
logger.info("Archivo leído: base_dups.csv con nrows=300. Shape: %s", df.shape)

# Vista previa ANTES de la primera agrupación
display(df.head(10))
logger.info("Preview inicial mostrada (10 filas). Tiempo: %.3fs", time.time()-t0)


C:\Users\eliez\AppData\Local\Temp\ipykernel_15276\2827886554.py:20: DtypeWarning: Columns (3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("base_dups.csv")
10:52:18 | INFO | Archivo leído: base_dups.csv con nrows=300. Shape: (354476, 12)


,f_creacion,f_entrega,centro,of_vta,keyacc,ruta_vta,cliente,canal_pedido,sector,pedido,material,cf
0,2025-08-25,2025-08-26,950,2320,99999,CXP1,500357957,YHH1,Z0,238793599,120112,3.0
1,2025-08-25,2025-08-26,950,2320,99999,CXP5,501148330,YHH1,Z0,238793836,120658,1.0
2,2025-08-25,2025-08-26,950,2320,99999,CXP3,500359865,YHHC,Z0,238793857,122615,1.0
3,2025-08-25,2025-08-26,950,2320,99999,CXP5,500442632,YHH1,Z0,238794905,122283,1.0
4,2025-08-25,2025-08-26,950,2320,99999,CXP5,500442632,YHH1,Z0,238794905,120524,2.0
5,2025-08-25,2025-08-26,950,2320,99999,CXP5,500442632,YHH1,Z0,238794905,120437,1.0
6,2025-08-25,2025-08-26,950,2320,99999,CXP5,500442632,YHH1,Z0,238794905,120121,1.0
7,2025-08-25,2025-08-26,950,2320,99999,CXP3,500676375,YHHB,Z0,238795237,123437,2.0
8,2025-08-25,2025-08-26,950,2320,99999,CXP3,500676375,YHHB,Z0,238795237,122285,1.0
9,2025-08-25,2025-08-26,950,2320,99999,CXP3,500676375,YHHB,Z0,238795237,122225,4.0


10:52:18 | INFO | Preview inicial mostrada (10 filas). Tiempo: 0.384s


In [2]:
# --- Celda 2 ---
import numpy as np
import logging, time, math
logger = logging.getLogger(__name__)

def similarity1(skus1,qtys1,skus2,qtys2):
    unique_skus = list(set(skus1+skus2))
    skus_v1 = [1 if sku in skus1 else 0 for sku in unique_skus]
    skus_v2 = [1 if sku in skus2 else 0 for sku in unique_skus]
    qtys_v1 = [qtys1[skus1.index(sku)] if sku in skus1 else 0 for sku in unique_skus]
    qtys_v2 = [qtys2[skus2.index(sku)] if sku in skus2 else 0 for sku in unique_skus]
    sku_similarity = np.dot(skus_v1, skus_v2)/(np.linalg.norm(skus_v1)*np.linalg.norm(skus_v2))
    qty_similarity = np.dot(qtys_v1, qtys_v2)/(np.linalg.norm(qtys_v1)*np.linalg.norm(qtys_v2))
    similarity = (sku_similarity + qty_similarity) / 2
    return similarity

def duplicidad1(x):
    N = len(x.pedido)
    if N<2: return [{}]
    dups = []
    for i in range(N):
        dups.append({
            x.pedido[j+i+1]: similarity1(x.material[i], x.cf[i], x.material[j+i+1], x.cf[j+i+1])
            for j in range(N-i-1)
            if similarity1(x.material[i], x.cf[i], x.material[j+i+1], x.cf[j+i+1]) > .5
        })
    return dups


In [3]:
# --- Celda 3 ---
import time, pandas as pd

t0 = time.time()
logger.info("Iniciando primera agrupación: ['f_entrega','cliente','pedido']")

# Validación mínima (sin renombrar)
required = {'f_entrega','cliente','pedido','material','cf'}
faltantes = required - set(df.columns)
if faltantes:
    raise KeyError(f"Faltan columnas requeridas en df: {faltantes}")

# Tipos mínimos seguros
df1 = df.copy()
df1['cliente'] = df1['cliente'].astype(str)
df1['pedido']  = df1['pedido'].astype(str)
df1['cf'] = pd.to_numeric(df1['cf'], errors='coerce').fillna(0)

# Primera agrupación: pedido único -> listas de materiales y cantidades (cf)
base = (
    df1.groupby(['f_entrega','cliente','pedido'], sort=False)
       .agg({'material': list, 'cf': list})
       .reset_index()
)

# Orden de vista
base.sort_values(['f_entrega','cliente'], ascending=False, inplace=True, kind='mergesort')

logger.info("Primera agrupación completada. Shape: %s | Tiempo: %.3fs", base.shape, time.time()-t0)
display(base.head(10))
logger.info("Preview de 'base' mostrada (10 filas).")


10:49:49 | INFO | Iniciando primera agrupación: ['f_entrega','cliente','pedido']
10:49:51 | INFO | Primera agrupación completada. Shape: (44600, 5) | Tiempo: 1.454s


,f_entrega,cliente,pedido,material,cf
41245,2025-08-27,501282421,238790628,[122406],[4.5]
32721,2025-08-27,501282419,238791888,"[129046, 120121, 120480, 125655, 122905]","[6.0, 6.0, 6.0, 3.0, 6.0]"
32720,2025-08-27,501282410,238791887,"[127271, 125547, 120524, 129041, 129035, 12790...","[60.0, 2.5, 5.0, 30.0, 30.0, 2.5, 30.0, 4.9999..."
32695,2025-08-27,501282403,238823542,"[120524, 120474, 120658, 121567, 120477, 12137...","[22.6666666666667, 5.666666666666668, 17.0, 5...."
32853,2025-08-27,501282400,238707209,[125674],[6.0]
32541,2025-08-27,501282394,238707207,"[125674, 120250, 121350, 120711, 10020792, 120...","[33.0, 22.00000000000001, 5.5, 2.75, 11.0, 5.5]"
32479,2025-08-27,501282391,238707206,"[127271, 120438, 125535, 120461, 10020724, 120...","[32.0, 32.0, 32.0, 32.0, 16.0, 31.999999999999..."
41234,2025-08-27,501282390,238758944,"[127070, 126214, 10020676, 125556, 125588]","[7.0, 7.0, 14.0, 7.0, 7.0]"
35132,2025-08-27,501282389,238758943,"[120250, 10017951]","[4.0, 8.0]"
35325,2025-08-27,501282386,238814431,"[120149, 129020]","[5.0, 5.0]"


10:49:51 | INFO | Preview de 'base' mostrada (10 filas).


In [4]:
# --- Celda 4A ---
t0 = time.time()
logger.info("Generando pares (i<j) por (f_entrega, cliente) a partir de 'base'...")

# Índice de pedido dentro del grupo (f_entrega, cliente)
base2 = base.copy()
base2['i'] = base2.groupby(['f_entrega','cliente']).cumcount()

# Self-join por clave de grupo y quedarnos con i_i < i_j (evita repetidos y autocomparación)
pairs = (
    base2.merge(base2, on=['f_entrega','cliente'], suffixes=('_i','_j'))
         .query('i_i < i_j')
         [['f_entrega','cliente','i_i','i_j','pedido_i','pedido_j','material_i','cf_i','material_j','cf_j']]
)

logger.info("Tabla de pares lista. Shape: %s | Tiempo: %.3fs", pairs.shape, time.time()-t0)
display(pairs.head(10))


10:49:56 | INFO | Generando pares (i<j) por (f_entrega, cliente) a partir de 'base'...
10:49:57 | INFO | Tabla de pares lista. Shape: (37310, 10) | Tiempo: 0.134s


,f_entrega,cliente,i_i,i_j,pedido_i,pedido_j,material_i,cf_i,material_j,cf_j
67,2025-08-27,501282228,0,1,238760578,238814400,[120447],[18.0],[120250],[15.0]
92,2025-08-27,501282179,0,1,238808332,238806064,"[120250, 120770]","[79.99999999999994, 30.0]","[120640, 121350]","[40.00000000000001, 39.99999999999997]"
190,2025-08-27,501281686,0,1,238778024,238778025,"[121567, 124224, 127932, 120524]","[2.0, 2.0, 2.0, 12.0]",[120477],[9.0]
244,2025-08-27,501279458,0,1,238822274,238809520,"[129035, 120603, 129020, 129036, 120371, 12904...","[9.0, 27.0, 18.0, 18.0, 9.0, 9.0, 18.0]","[120371, 121572, 129036, 129035, 129020, 12904...","[8.0, 24.0, 16.0, 8.0, 16.0, 8.0, 24.0]"
258,2025-08-27,501278595,0,1,238790710,238788497,"[122915, 122727, 122627, 122236, 120437, 12025...","[1.0, 1.0, 1.0, 1.0, 1.0, 3.0, 1.0, 1.0, 1.0, ...","[120802, 120377, 120373, 120378]","[2.0, 1.0, 1.0, 2.0]"
268,2025-08-27,501278225,0,1,238802618,238801518,"[123435, 123433, 121325, 120710, 120696, 12066...","[1.0, 2.0, 2.0, 4.0, 1.0, 1.0, 1.0, 1.0, 2.0, ...","[124314, 124214, 124204, 123437]","[1.0, 1.0, 1.0, 1.0]"
284,2025-08-27,501277593,0,1,238797150,238797520,"[120437, 120367, 120373, 120438, 122202]","[27.0, 9.0, 9.0, 18.0, 36.0]","[122906, 122715, 122615, 120657, 124214, 12295...","[24.0, 24.0, 36.0, 24.0, 12.0, 12.0, 12.0]"
308,2025-08-27,501276736,0,1,238831022,238831041,"[127956, 127923, 127843, 128009, 127801]","[1.0, 1.0, 1.0, 1.0, 1.0]","[126214, 125659, 125647, 127070, 125631, 12555...","[6.0, 1.0, 1.0, 7.0, 6.0, 6.0, 10.0]"
309,2025-08-27,501276736,0,2,238831022,238831021,"[127956, 127923, 127843, 128009, 127801]","[1.0, 1.0, 1.0, 1.0, 1.0]","[125231, 122232, 122200, 121378, 120760, 12052...","[4.0, 2.0, 2.0, 3.0, 1.0, 4.0, 1.0, 4.0, 3.0, ..."
310,2025-08-27,501276736,0,3,238831022,238827614,"[127956, 127923, 127843, 128009, 127801]","[1.0, 1.0, 1.0, 1.0, 1.0]","[127443, 127354, 127340, 127294, 127394]","[6.0, 10.0, 10.0, 5.0, 10.0]"


In [5]:
# --- Celda 4B ---
t0 = time.time()
logger.info("Calculando similitud por par con 'similarity1' (original)...")

pairs['similarity'] = pairs.apply(
    lambda r: similarity1(r['material_i'], r['cf_i'], r['material_j'], r['cf_j']),
    axis=1
)

logger.info("Similaridad calculada. Tiempo: %.3fs", time.time()-t0)
display(pairs.head(10))


10:50:02 | INFO | Calculando similitud por par con 'similarity1' (original)...
C:\Users\eliez\AppData\Local\Temp\ipykernel_24484\1691442861.py:13: RuntimeWarning: invalid value encountered in scalar divide
  qty_similarity = np.dot(qtys_v1, qtys_v2)/(np.linalg.norm(qtys_v1)*np.linalg.norm(qtys_v2))
10:50:04 | INFO | Similaridad calculada. Tiempo: 1.840s


,f_entrega,cliente,i_i,i_j,pedido_i,pedido_j,material_i,cf_i,material_j,cf_j,similarity
67,2025-08-27,501282228,0,1,238760578,238814400,[120447],[18.0],[120250],[15.0],0.000000
92,2025-08-27,501282179,0,1,238808332,238806064,"[120250, 120770]","[79.99999999999994, 30.0]","[120640, 121350]","[40.00000000000001, 39.99999999999997]",0.000000
190,2025-08-27,501281686,0,1,238778024,238778025,"[121567, 124224, 127932, 120524]","[2.0, 2.0, 2.0, 12.0]",[120477],[9.0],0.000000
244,2025-08-27,501279458,0,1,238822274,238809520,"[129035, 120603, 129020, 129036, 120371, 12904...","[9.0, 27.0, 18.0, 18.0, 9.0, 9.0, 18.0]","[120371, 121572, 129036, 129035, 129020, 12904...","[8.0, 24.0, 16.0, 8.0, 16.0, 8.0, 24.0]",0.992764
258,2025-08-27,501278595,0,1,238790710,238788497,"[122915, 122727, 122627, 122236, 120437, 12025...","[1.0, 1.0, 1.0, 1.0, 1.0, 3.0, 1.0, 1.0, 1.0, ...","[120802, 120377, 120373, 120378]","[2.0, 1.0, 1.0, 2.0]",0.000000
268,2025-08-27,501278225,0,1,238802618,238801518,"[123435, 123433, 121325, 120710, 120696, 12066...","[1.0, 2.0, 2.0, 4.0, 1.0, 1.0, 1.0, 1.0, 2.0, ...","[124314, 124214, 124204, 123437]","[1.0, 1.0, 1.0, 1.0]",0.000000
284,2025-08-27,501277593,0,1,238797150,238797520,"[120437, 120367, 120373, 120438, 122202]","[27.0, 9.0, 9.0, 18.0, 36.0]","[122906, 122715, 122615, 120657, 124214, 12295...","[24.0, 24.0, 36.0, 24.0, 12.0, 12.0, 12.0]",0.000000
308,2025-08-27,501276736,0,1,238831022,238831041,"[127956, 127923, 127843, 128009, 127801]","[1.0, 1.0, 1.0, 1.0, 1.0]","[126214, 125659, 125647, 127070, 125631, 12555...","[6.0, 1.0, 1.0, 7.0, 6.0, 6.0, 10.0]",0.000000
309,2025-08-27,501276736,0,2,238831022,238831021,"[127956, 127923, 127843, 128009, 127801]","[1.0, 1.0, 1.0, 1.0, 1.0]","[125231, 122232, 122200, 121378, 120760, 12052...","[4.0, 2.0, 2.0, 3.0, 1.0, 4.0, 1.0, 4.0, 3.0, ...",0.000000
310,2025-08-27,501276736,0,3,238831022,238827614,"[127956, 127923, 127843, 128009, 127801]","[1.0, 1.0, 1.0, 1.0, 1.0]","[127443, 127354, 127340, 127294, 127394]","[6.0, 10.0, 10.0, 5.0, 10.0]",0.000000


In [6]:
# --- Celda 5 (final, robusta): df_final + dups directo + merge seguro ---
UMBRAL = 0.5
t0 = time.time()

# 1) df_final plano (pares con similitud > umbral)
df_final = (
    pairs.loc[pairs['similarity'] > UMBRAL,
              ['f_entrega','cliente','pedido_i','pedido_j','similarity']]
         .sort_values(['f_entrega','cliente','similarity'], ascending=[True, True, False])
         .reset_index(drop=True)
)
display(df_final.head(10))
logger.info("df_final listo (umbral=%.2f). Shape: %s", UMBRAL, df_final.shape)

# 2) Normalizar llaves (para evitar descalces en merge)
def _norm_keys(df_):
    df_ = df_.copy()
    df_['f_entrega'] = pd.to_datetime(df_['f_entrega'], errors='coerce').dt.strftime('%Y-%m-%d')
    df_['cliente'] = df_['cliente'].astype(str).str.strip()
    return df_

base_norm    = _norm_keys(base)
df_final_norm = _norm_keys(df_final)

# 3) Construir 'dups' por grupo usando named aggregation (evita expansión de columnas)
def _build_dups_list_from_group_idx(idx, ndigits=4):
    # idx: índices de df_final_norm que pertenecen al grupo
    g = df_final_norm.loc[idx, ['pedido_i','pedido_j','similarity']]
    # lista de tuplas (pedido_i, pedido_j, similarity_redondeada)
    return [(a, b, round(float(s), ndigits))
            for a, b, s in g.itertuples(index=False, name=None)]

if not df_final_norm.empty:
    dups_by_group = (
        df_final_norm
        .groupby(['f_entrega','cliente'])
        .agg(dups=('similarity', lambda s: _build_dups_list_from_group_idx(s.index)))
        .reset_index()
    )
else:
    dups_by_group = pd.DataFrame(columns=['f_entrega','cliente','dups'])

logger.info("dups_by_group construido. Shape: %s", dups_by_group.shape)

# 4) Reconstruir basegr y mergear 'dups'
basegr = (
    base_norm
    .groupby(['f_entrega','cliente'], sort=False)
    .agg({'pedido':list,'material':list,'cf':list})
    .reset_index()
    .merge(dups_by_group, on=['f_entrega','cliente'], how='left')
)

# 5) Formatear columnas y mostrar SOLO grupos con duplicados
basegr['dups'] = basegr['dups'].apply(lambda x: x if isinstance(x, list) else [])
basegr['dups_str'] = basegr['dups'].apply(
    lambda ps: "; ".join([f"{a} ↔ {b} ({s})" for a,b,s in ps]) if ps else ""
)

gr_con_dups = (
    basegr[basegr['dups'].str.len() > 0]
      .sort_values(['f_entrega','cliente'])
      .reset_index(drop=True)
)

if not gr_con_dups.empty:
    display(gr_con_dups[['f_entrega','cliente','pedido','dups','dups_str']].head(20))
    print(f"Grupos con duplicados detectados: {gr_con_dups.shape[0]}")
else:
    print("⚠️ No se detectaron grupos con duplicados (verifica que existan llaves comunes entre base y df_final).")

logger.info("basegr con 'dups' directo desde df_final. Tiempo total: %.3fs", time.time()-t0)


,f_entrega,cliente,pedido_i,pedido_j,similarity
0,2025-08-25,500265488,238752083,238747373,0.943535
1,2025-08-25,500267558,238776004,238775983,0.696741
2,2025-08-25,500269458,238770807,238771243,0.517154
3,2025-08-25,500270637,238763642,238764477,0.585640
4,2025-08-25,500270692,238765442,238762553,0.586133
5,2025-08-25,500271690,238770545,238773943,0.797724
6,2025-08-25,500273245,238771442,238770251,0.912871
7,2025-08-25,500273355,238771921,238770204,0.748753
8,2025-08-25,500273355,238770961,238770703,0.739679
9,2025-08-25,500274828,238768982,238769674,0.919389


10:50:09 | INFO | df_final listo (umbral=0.50). Shape: (2333, 5)
10:50:10 | INFO | dups_by_group construido. Shape: (789, 3)


,f_entrega,cliente,pedido,dups,dups_str
0,2025-08-25,500265488,"[238752083, 238747373]","[(238752083, 238747373, 0.9435)]",238752083 ↔ 238747373 (0.9435)
1,2025-08-25,500267558,"[238776004, 238775983]","[(238776004, 238775983, 0.6967)]",238776004 ↔ 238775983 (0.6967)
2,2025-08-25,500269458,"[238770807, 238771243]","[(238770807, 238771243, 0.5172)]",238770807 ↔ 238771243 (0.5172)
3,2025-08-25,500270637,"[238763730, 238763642, 238764477, 238761452, 2...","[(238763642, 238764477, 0.5856)]",238763642 ↔ 238764477 (0.5856)
4,2025-08-25,500270692,"[238765442, 238759268, 238763035, 238762553, 2...","[(238765442, 238762553, 0.5861)]",238765442 ↔ 238762553 (0.5861)
5,2025-08-25,500271690,"[238770545, 238773943]","[(238770545, 238773943, 0.7977)]",238770545 ↔ 238773943 (0.7977)
6,2025-08-25,500273245,"[238771843, 238771442, 238770251]","[(238771442, 238770251, 0.9129)]",238771442 ↔ 238770251 (0.9129)
7,2025-08-25,500273355,"[238771921, 238770204, 238770961, 238770703]","[(238771921, 238770204, 0.7488), (238770961, 2...",238771921 ↔ 238770204 (0.7488); 238770961 ↔ 23...
8,2025-08-25,500274828,"[238768982, 238769674]","[(238768982, 238769674, 0.9194)]",238768982 ↔ 238769674 (0.9194)
9,2025-08-25,500277468,"[238736997, 238746475]","[(238736997, 238746475, 1.0)]",238736997 ↔ 238746475 (1.0)


10:50:11 | INFO | basegr con 'dups' directo desde df_final. Tiempo total: 2.192s


Grupos con duplicados detectados: 789
